In [1]:
import datetime
start_time = datetime.datetime.now(); print(start_time)

2018-12-01 18:39:56.430394


In [2]:
%matplotlib inline

In [3]:
secret = False #Will be put to True for a CASD export

In [4]:
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd ; pd.set_option("display.max_columns",1000)
import pickle

In [5]:
import pickle
pickle_year_variable_path = "C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Program/Program_only_married_couples/year_variable/"
year =  pickle.load(open(pickle_year_variable_path+"year_variable.p",'rb')); print(year)

2011


In [6]:
store_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/hdf/edp_2016_final.h5'
reaction_store_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/hdf/reaction.h5'
pickles_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/pickles/{}'.format(year)

In [7]:
result_path = u"C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Program/Program_only_married_couples/hdf_married_couples/"

In [8]:
df = pd.read_hdf(result_path+"married_couples_step_4.h5", "fiscr")

We define the taxable income (Revenu Brut Global) based on the 7 income categories present in the database.

In [9]:
df["Rbg_2011"] =(( df.ZTSAM_2011*0.9) + df.ZRAGM_2011 + df.ZRACM_2011 + df.ZRNCM_2011 + df.ZFONM_2011 + df.ZPERM_2011*0.9
                    + df.ZRICM_2011)
df["Rbg_2014"] =(( df.ZTSAM_2014*0.9) + df.ZRAGM_2014 + df.ZRACM_2014 + df.ZRNCM_2014 + df.ZFONM_2014 + df.ZPERM_2014*0.9
                    + df.ZRICM_2014)
df["Delta_Rbg"] = df.Rbg_2014 - df.Rbg_2011
df["Delta_rbg"] = df.Rbg_2014 - df.Rbg_2011
df["Delta_ZIMPOM"] = df.ZIMPOM_DECL_2014 - df.ZIMPOM_2011

In [10]:
threshold_byr_rbg = pd.DataFrame()
threshold_byr_rbg["2011"] = pd.Series([63233,73516,94451,115185,135941,150684])
threshold_byr_rbg["2014"] = pd.Series([58291,63530,73806,84103,94368,104633])

In [11]:
all_thresholds = pd.concat([threshold_byr_rbg["2011"], threshold_byr_rbg["2014"]]).sort_values()
thrsh_bool_list = ["Threshold_{}".format(thrsh) for thrsh in  all_thresholds ]
for thrsh in all_thresholds:
    df["Threshold_{}".format(thrsh)] = df.Rbg_2011>= thrsh

In [12]:
for nb_enf in range(1,7): ### in modification
    df["Between_threshold_{}_child".format(nb_enf)
          ] = ((df.Rbg_2011>  threshold_byr_rbg["2014"].iloc[nb_enf-1])
               & (df.Rbg_2011< threshold_byr_rbg["2011"].iloc[nb_enf-1]))


In [13]:

for nb_enf in range(1,7): ### in modification
    df["Over_threshold_{}_child".format(nb_enf)
          ] =  (df.Rbg_2011 > threshold_byr_rbg["2011"].iloc[nb_enf-1])
for nb_enf in range(1,7): ### in modification
    df["Below_thresholds_{}_child".format(nb_enf)
          ] =  (df.Rbg_2011 < threshold_byr_rbg["2011"].iloc[nb_enf-1])

In [14]:
df["Over_threshold"] = False
df["Between_thresholds"] = False
for child in range(1,7): ### in modification
    df["Over_threshold"] +=  (df["Over_threshold_{}_child".format(child)] & (df["NBPAC_2011"] ==child))
    df["Between_thresholds"] +=  (df["Between_threshold_{}_child".format(child)] & (df["NBPAC_2011"] ==child))

C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-3.6.3.0Qt5\python-3.6.3.amd64\lib\site-packages\pandas\core\computation\expressions.py:179: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


In [15]:
## Interactions hardcoded
for child in range(1,7):
    df["Over_{}_X_{}_child".format(child,child)] = df["Over_threshold_{}_child".format(child)] * (df["NBPAC_2011"]==child)
    df["Between_{}_X_{}_child".format(child,child)] = df["Between_threshold_{}_child".format(child)] * (df["NBPAC_2011"]==child)

C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-3.6.3.0Qt5\python-3.6.3.amd64\lib\site-packages\pandas\core\computation\expressions.py:179: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


In [16]:
df["Intercept"] = 1 
#df["Rbg_splines"]  = pd.cut(df.Rbg_2011, 100)

In [17]:
for child in range(0,7):
    df["child_{}".format(child)] = df.NBPAC_2011 == child

In [18]:
interactions_rbg_between = ["Interactions_rbg_child{}_between".format(child) for child in range(1,7)]
interactions_rbg_over = ["Interactions_rbg_child{}_over".format(child) for child in range(1,7)]
interactions_child_over = ["Interactions_child{}_over".format(child) for child in range(1,7)]




df["Interactions_rbg_child1_between"] = df.eval("Delta_rbg*child_1*Between_threshold_1_child")
df["Interactions_rbg_child2_between"] = df.eval("Delta_rbg*child_2*Between_threshold_2_child")
df["Interactions_rbg_child3_between"] = df.eval("Delta_rbg*child_3*Between_threshold_3_child")
df["Interactions_rbg_child4_between"] = df.eval("Delta_rbg*child_4*Between_threshold_4_child")
df["Interactions_rbg_child5_between"] = df.eval("Delta_rbg*child_5*Between_threshold_5_child")
df["Interactions_rbg_child6_between"] = df.eval("Delta_rbg*child_6*Between_threshold_6_child")



df["Interactions_rbg_child1_over"] = df.eval("Delta_rbg*child_1*Over_threshold_1_child")
df["Interactions_rbg_child2_over"] = df.eval("Delta_rbg*child_2*Over_threshold_2_child")
df["Interactions_rbg_child3_over"] = df.eval("Delta_rbg*child_3*Over_threshold_3_child")
df["Interactions_rbg_child4_over"] = df.eval("Delta_rbg*child_4*Over_threshold_4_child")
df["Interactions_rbg_child5_over"] = df.eval("Delta_rbg*child_5*Over_threshold_5_child")
df["Interactions_rbg_child6_over"] = df.eval("Delta_rbg*child_6*Over_threshold_6_child")


df["Interactions_child1_over"] = (df["child_1"]* df["Over_threshold_1_child"]).astype("int")
df["Interactions_child2_over"] = (df["child_2"]* df["Over_threshold_2_child"]).astype("int")
df["Interactions_child3_over"] = (df["child_3"]* df["Over_threshold_3_child"]).astype("int")
df["Interactions_child4_over"] = (df["child_4"]* df["Over_threshold_4_child"]).astype("int")
df["Interactions_child5_over"] = (df["child_5"]* df["Over_threshold_5_child"]).astype("int")
df["Interactions_child6_over"] = (df["child_6"]* df["Over_threshold_6_child"]).astype("int")





df["Not_touched_by_reform"] = (df.NBPAC_2011 == 0).astype("int")
for nb_enf in range(1,7): ### in modification
    df["Not_touched_by_reform".format(nb_enf)
          ] +=((df.Rbg_2011<  threshold_byr_rbg["2014"].iloc[nb_enf-1])
                *( df.NBPAC_2011 == nb_enf)).astype("float")

df["Delta_Rbg_times_not_touched_reform"] = df.Not_touched_by_reform * df.Delta_rbg

C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-3.6.3.0Qt5\python-3.6.3.amd64\lib\site-packages\pandas\core\computation\expressions.py:179: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


In [19]:
hello = pd.DataFrame()

In [20]:
hello.consolidate()

C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-3.6.3.0Qt5\python-3.6.3.amd64\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: consolidate is deprecated and will be removed in a future release.
  """Entry point for launching an IPython kernel.


""


In [21]:
df.consolidate(inplace = True)

C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-3.6.3.0Qt5\python-3.6.3.amd64\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: consolidate is deprecated and will be removed in a future release.
  """Entry point for launching an IPython kernel.


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 789743 entries, 0 to 789742
Columns: 513 entries, ID_FISC_LOG_DIFF to Delta_Rbg_times_not_touched_reform
dtypes: bool(55), float64(231), int32(6), int64(127), object(94)
memory usage: 2.7+ GB


In [23]:
df.select_dtypes("category")

""
0
1
2
3
4
5
6
7
8
9


In [24]:
df.to_hdf(result_path+"married_couples_step_last.h5", "fiscr")

C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-3.6.3.0Qt5\python-3.6.3.amd64\lib\site-packages\pandas\core\generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block4_values] [items->['ID_FISC_LOG_DIFF', 'DEP_2011', 'DEPCOM_2011', 'I_CHAMPM_2011', 'I_PAUVRE50M_2011', 'I_PAUVRE60M_2011', 'DEP_2014', 'DEPCOM_2014', 'I_PAUVRE50M_2014', 'I_PAUVRE60M_2014', 'TYPE_FISC_2011', 'TYPE_FISC_2014', 'ID_DIFF_father', 'ID_DIFF_mother', 'DEP_2011_to_drop', 'SECCAD_2011', 'OCC_2011', 'DNATLC_2011', 'NATLOC_2011', 'TAX_2011', 'NLOC4_2011', 'EAU_2011', 'EGOUT_2011', 'ELEC_2011', 'GAZ_2011', 'CHAUFF_2011', 'ASCENSEUR_2011', 'DEP_2014_to_drop', 'SECCAD_2014', 'OCC_2014', 'DNATLC_2014', 'NATLOC_2014', 'TAX_2014', 'NLOC4_2014', 'EAU_2014', 'EGOUT_2014', 'ELEC_2014', 'GAZ_2014', 'CHAUFF_2014', 'ASCENSEUR_2014', 'NUM_IRIS_2014', 'NUM_QPV_2014', 'DEPR_2014_father', 'COMR_2014_father'

In [25]:
stop_time = datetime.datetime.now(); print("now: ",stop_time);
execution_time = stop_time - start_time; print("execution_time: ",execution_time)

now:  2018-12-01 18:40:32.844704
execution_time:  0:00:36.414310
